# Description

# Setup

In [ ]:
import pandas as pd
import requests
import plotly.express as px
from ons_api import get_list_of_datasets, get_dataset_by_name, get_edition, get_timeseries
import io

# Data

## BR

In [3]:
url = "https://api.bcb.gov.br/dados/serie/bcdata.sgs.433/dados?formato=csv&dataInicial=01/01/1995&dataFinal=31/12/2025"
df = pd.read_csv(url, sep=";", decimal=",", date_format="%d/%m/%Y", parse_dates=[0])
df.head()

,data,valor
0,1995-01-01,1.70
1,1995-02-01,1.02
2,1995-03-01,1.55
3,1995-04-01,2.43
4,1995-05-01,2.67


## UK

In [30]:
dss = get_list_of_datasets()
dataset_name = "Consumer Prices Index including owner occupiers' housing costs (CPIH)"
dataset = get_dataset_by_name(dss, dataset_name)
edition_url = get_edition(dataset)
r = requests.get(edition_url)
results = r.json()
latest_edition_url = results.get("downloads").get("csv").get("href")
latest_edition_url
r = requests.get(latest_edition_url).content
df_uk = pd.read_csv(io.BytesIO(r), date_format="%b-%y", parse_dates=[1,2])
df_uk.head()

,v4_0,mmm-yy,Time,uk-only,Geography,cpih1dim1aggid,Aggregate
0,147.1,2025-05-01,2025-05-01,K02000001,United Kingdom,CP0122,"01.2.2 Mineral waters, soft drinks and juices"
1,138.9,2025-05-01,2025-05-01,K02000001,United Kingdom,CP051,"05.1 Furniture, furnishings and carpets"
2,138.7,2025-05-01,2025-05-01,K02000001,United Kingdom,CP0721,07.2.1 Spare parts and accessories
3,162.9,2025-05-01,2025-05-01,K02000001,United Kingdom,CP0733,07.3.3 Passenger transport by air
4,175.4,2025-05-01,2025-05-01,K02000001,United Kingdom,CP081,08.1 Postal services


In [27]:
df_uk['Aggregate'].sort_values().drop_duplicates()

45306    01 Food and non-alcoholic beverages
3307                               01.1 Food
34823               01.1.1 Bread and cereals
38210                            01.1.2 Meat
37455                            01.1.3 Fish
                        ...                 
2694                 12.5.3 Health insurance
16563             12.5.4 Transport insurance
12955          12.6 Financial services (nec)
36575              12.7 Other services (nec)
2774                           Overall Index
Name: Aggregate, Length: 126, dtype: object

In [31]:
df_uk[df_uk['Aggregate']=="Overall Index"]

,v4_0,mmm-yy,Time,uk-only,Geography,cpih1dim1aggid,Aggregate
101,138.0,2025-05-01,2025-05-01,K02000001,United Kingdom,CP00,Overall Index
166,137.7,2025-04-01,2025-04-01,K02000001,United Kingdom,CP00,Overall Index
284,136.1,2025-03-01,2025-03-01,K02000001,United Kingdom,CP00,Overall Index
466,135.6,2025-02-01,2025-02-01,K02000001,United Kingdom,CP00,Overall Index
506,135.1,2025-01-01,2025-01-01,K02000001,United Kingdom,CP00,Overall Index
...,...,...,...,...,...,...,...
54252,48.0,1988-05-01,1988-05-01,K02000001,United Kingdom,CP00,Overall Index
54337,47.8,1988-04-01,1988-04-01,K02000001,United Kingdom,CP00,Overall Index
54442,47.2,1988-03-01,1988-03-01,K02000001,United Kingdom,CP00,Overall Index
54578,47.0,1988-02-01,1988-02-01,K02000001,United Kingdom,CP00,Overall Index


In [34]:
data_chart = df_uk[df_uk['Aggregate']=="Overall Index"]

fig = px.line(
    data_chart,
    x='Time',
    y='v4_0',
    width=700,
    template='none'
)
fig 

## World Bank

In [38]:
from zipfile import ZipFile

In [49]:
cpi_source = 'https://api.worldbank.org/v2/en/indicator/FP.CPI.TOTL.ZG?downloadformat=csv'
r = requests.get(cpi_source).content
zip_file = ZipFile(io.BytesIO(r))
zip_file.infolist()
pd.read_csv(zip_file.open("API_FP.CPI.TOTL.ZG_DS2_en_csv_v2_80938.csv"), 
            encoding='ISO-8859-1', 
            skiprows=4)

,Country Name,Country Code,Indicator Name,Indicator Code,1960,1961,1962,1963,1964,1965,...,2016,2017,2018,2019,2020,2021,2022,2023,2024,Unnamed: 69
0,Aruba,ABW,"Inflation, consumer prices (annual %)",FP.CPI.TOTL.ZG,NaN,NaN,NaN,NaN,NaN,NaN,...,-0.931196,-1.028282,3.626041,4.257462,NaN,NaN,NaN,NaN,NaN,NaN
1,Africa Eastern and Southern,AFE,"Inflation, consumer prices (annual %)",FP.CPI.TOTL.ZG,NaN,NaN,NaN,NaN,NaN,NaN,...,6.596505,6.399343,4.720805,4.644967,5.405162,7.240978,10.773751,7.126975,NaN,NaN
2,Afghanistan,AFG,"Inflation, consumer prices (annual %)",FP.CPI.TOTL.ZG,NaN,NaN,NaN,NaN,NaN,NaN,...,4.383892,4.975952,0.626149,2.302373,5.601888,5.133203,13.712102,-4.644709,-6.601186,NaN
3,Africa Western and Central,AFW,"Inflation, consumer prices (annual %)",FP.CPI.TOTL.ZG,NaN,NaN,NaN,NaN,NaN,NaN,...,1.487416,1.725486,1.784050,1.983092,2.490378,3.745700,7.949251,5.221168,NaN,NaN
4,Angola,AGO,"Inflation, consumer prices (annual %)",FP.CPI.TOTL.ZG,NaN,NaN,NaN,NaN,NaN,NaN,...,30.694415,29.844480,19.628938,17.080954,22.271539,25.754295,21.355290,13.644102,28.240495,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
261,Kosovo,XKX,"Inflation, consumer prices (annual %)",FP.CPI.TOTL.ZG,NaN,NaN,NaN,NaN,NaN,NaN,...,0.273169,1.488234,1.053798,2.675992,0.198228,3.353691,11.580510,4.944324,1.619450,NaN
262,"Yemen, Rep.",YEM,"Inflation, consumer prices (annual %)",FP.CPI.TOTL.ZG,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
263,South Africa,ZAF,"Inflation, consumer prices (annual %)",FP.CPI.TOTL.ZG,1.288877,2.102351,1.246275,1.337971,2.534989,4.069019,...,6.602908,5.186187,4.509873,4.102851,3.232388,4.618745,7.039873,6.075244,4.361152,NaN
264,Zambia,ZMB,"Inflation, consumer prices (annual %)",FP.CPI.TOTL.ZG,NaN,NaN,NaN,NaN,NaN,NaN,...,17.869730,6.577312,7.494572,9.150316,15.733060,22.020768,10.993204,10.884532,NaN,NaN


# analysis

In [4]:
data_chart = df.copy()
data_chart['cum_ipca'] = ((1+data_chart['valor']/100).cumprod() - 1)
fig = px.line(
    data_chart,
    x='data',
    y='cum_ipca',
    width=700,
    template='none'
)
fig 